****TITANIC VERİ SETİNİN HOLDOUT YÖNTEMİYLE AYRILIP ÜZERİNDE SINIFLANDIRMA TAHMİN MODELİ KURULMASI****

Veri Seti Hakkında Bilgi:
Veri seti Titanic gemi kazasında bulunan kişilere ait bilgileri içermektedir.
768 gözlem ve 12 değişkenden oluşmaktadır.
Hedef değişken "Survived" olarak belirtilmiş olup;
1, kişinin hayatta kalmasını;
0 ise kişinin hayatını kaybetmesini belirtmektedir.

**Kaynak**: https://bootcamp.veribilimiokulu.com/egitim/veri-bilimci-yetistirme-programi/

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, \
    roc_auc_score, confusion_matrix, classification_report, plot_roc_curve

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 170)

In [ ]:
df=pd.read_csv("../input/testtitanic/titanic_data.csv")

In [ ]:
df.head()

In [ ]:
#Column isimlerini büyütüyoruz.
df.columns = [col.upper() for col in df.columns]

### Özellik Oluşturma

In [ ]:
#Bilet numaralarının uzunluğuna bakıyoruz.
df["TICKET_N"] = [len(i) for i in df["TICKET"]]

In [ ]:
#Bazı bilet numaralarında yalnızca sayı bazılarında ise hem sayı hem harf bulunmakta, bunun ayrımını 
# yapıyoruz.
for i in df["TICKET"]:
    if i.isdigit():
        df.loc[(df["TICKET"] == i), "TICKET_ASCII"] = 0
    else:
        df.loc[(df["TICKET"] == i), "TICKET_ASCII"] = 1

In [ ]:
 df["CABIN"].tail() #Görüldüğü gibi boş değerler bulunmakta.

In [ ]:
#Cabin numaralarının boş olma sebebi gemi mürettebatının cabin numarasının olmamasıymış,
#Bu yüzden buradaki boş değerler de bir anlam ifade ettiğinden yolcuların cabin numarasının 1 
# mürettebatınkinin ise 0 dönmesini sağlıyoruz.
df["NEW_CABIN_BOOL"] = df["CABIN"].notnull().astype('int')

In [ ]:
#İsimlerin uzunluğunu saydırıyoruz.
df["NEW_NAME_COUNT"] = df["NAME"].str.len()

In [ ]:
# satırlardaki toplam kelime sayısına bakıyoruz.
df["NEW_NAME_WORD_COUNT"] = df["NAME"].apply(lambda x: len(str(x).split(" ")))

In [ ]:
#Kişilerin unvanlarına bu regex koduyla ulaşıyoruz.
df.NAME.str.extract(' ([A-Za-z]+)\.', expand=False).head()

In [ ]:
df['NEW_TITLE'] = df.NAME.str.extract(' ([A-Za-z]+)\.', expand=False)

In [ ]:
#Yolcuların kaç kişi birlikte(yakınlarıyla) seyahat ettiğini sorguluyoruz.Yalnız da olabilirler.
df["NEW_FAMILY_SIZE"] = df["SIBSP"] + df["PARCH"] + 1

In [ ]:
df.loc[((df['SIBSP'] + df['PARCH']) > 0), "NEW_IS_ALONE"] = "NO"
df.loc[((df['SIBSP'] + df['PARCH']) == 0), "NEW_IS_ALONE"] = "YES"

In [ ]:
# yaşları kategorik hale getiriyoruz,genç olgun yaşlı gibi
df.loc[(df['AGE'] < 18), 'NEW_AGE_CAT'] = 'young'
df.loc[(df['AGE'] >= 18) & (df['AGE'] < 56), 'NEW_AGE_CAT'] = 'mature'
df.loc[(df['AGE'] >= 56), 'NEW_AGE_CAT'] = 'senior'

In [ ]:
# yaş ile cinsiyeti birlikte değerlendirmek için bu kodları yazıyoruz.
df.loc[(df['SEX'] == 'male') & (df['NEW_AGE_CAT'] == 'young'), 'NEW_SEX_CAT'] = 'youngmale'
df.loc[(df['SEX'] == 'male') & (df['NEW_AGE_CAT'] == 'mature'), 'NEW_SEX_CAT'] = 'maturemale'
df.loc[(df['SEX'] == 'male') & (df['NEW_AGE_CAT'] == 'senior'), 'NEW_SEX_CAT'] = 'seniormale'
df.loc[(df['SEX'] == 'female') & (df['NEW_AGE_CAT'] == 'young'), 'NEW_SEX_CAT'] = 'youngfemale'
df.loc[(df['SEX'] == 'female') & (df['NEW_AGE_CAT'] == 'mature'), 'NEW_SEX_CAT'] = 'maturefemale'
df.loc[(df['SEX'] == 'female') & (df['NEW_AGE_CAT'] == 'senior'), 'NEW_SEX_CAT'] = 'seniorfemale'

In [ ]:
def grab_col_names(dataframe, cat_th=10, car_th=20):    
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car

In [ ]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)
num_cols

In [ ]:
def nums_cols_new(df, num_cols):
    a = df.shape[0]
    for col in num_cols:
        if (df.loc[((a) - 1, col)] - df.loc[0, col]) / (a - 1) == 1:
            print(col)
            num_cols.remove(col)
    return num_cols

In [ ]:
num_cols = nums_cols_new(df, num_cols)

In [ ]:
num_cols

### Aykırı Değerleri Baskılama

In [ ]:
#Age değişkeninin quantile çeyreklik değerlerine bakıyoruz.
q1 = df["AGE"].quantile(0.25)
q3 = df["AGE"].quantile(0.75)
iqr = q3 - q1
up = q3 + 1.5 * iqr
low = q1 - 1.5 * iqr

In [ ]:
#Aykırı değerleri baskılıyoruz.
df.loc[(df["AGE"] < low), "AGE"] = low
df.loc[(df["AGE"] > up), "AGE"] = up

### Eksik Değerler

In [ ]:
#Eksik değeri olan tüm columnları bu şekilde elde ediyoruz.
na_col = [col for col in df.columns if df[col].isnull().sum() > 0]

In [ ]:
#En çok hangi columnda eksik değer var ona bakıp sıralıyoruz.
df[na_col].isnull().sum().sort_values(ascending=False)
n_miss = df[na_col].isnull().sum().sort_values(ascending=False)

In [ ]:
n_miss

In [ ]:
#Bu columnlardan istediğimiz bilgileri aldık artık siliyoruz.
remove_cols = ["TICKET", "NAME", "CABIN"]
df.drop(remove_cols, inplace=True, axis=1)

In [ ]:
df.isnull().sum()

In [ ]:
# Age değişkenindeki null değerleri unvanlarının yaş ortalamasına göre dolduruyoruz.
df["AGE"] = df["AGE"].fillna(df.groupby("NEW_TITLE")["AGE"].transform("median"))

In [ ]:
# tipi object(kategorik değişken) olan ve eşsiz en fazla 10 tane değeri olanların null değerlerini mode'larıyla dolduruyoruz.
df = df.apply(lambda x: x.fillna(x.mode()[0]) if (x.dtype == "O" and len(x.unique()) <= 10) else x, axis=0)

In [ ]:
df.isnull().sum().sum()

In [ ]:
#Age değerlerini tekrar kategorize ediyoruz, eksik değerleri doldurduğumuz için bu işlemi tekrar yapıyoruz.
df["NEW_AGE_PCLASS"] = df["AGE"] * df["PCLASS"]
df.loc[(df['AGE'] < 18), 'NEW_AGE_CAT'] = 'young'
df.loc[(df['AGE'] >= 18) & (df['AGE'] < 56), 'NEW_AGE_CAT'] = 'mature'
df.loc[(df['AGE'] >= 56), 'NEW_AGE_CAT'] = 'senior'

In [ ]:
df.loc[(df['SEX'] == 'male') & (df['NEW_AGE_CAT'] == 'young'), 'NEW_SEX_CAT'] = 'youngmale'
df.loc[(df['SEX'] == 'male') & (df['NEW_AGE_CAT'] == 'mature'), 'NEW_SEX_CAT'] = 'maturemale'
df.loc[(df['SEX'] == 'male') & (df['NEW_AGE_CAT'] == 'senior'), 'NEW_SEX_CAT'] = 'seniormale'
df.loc[(df['SEX'] == 'female') & (df['NEW_AGE_CAT'] == 'young'), 'NEW_SEX_CAT'] = 'youngfemale'
df.loc[(df['SEX'] == 'female') & (df['NEW_AGE_CAT'] == 'mature'), 'NEW_SEX_CAT'] = 'maturefemale'
df.loc[(df['SEX'] == 'female') & (df['NEW_AGE_CAT'] == 'senior'), 'NEW_SEX_CAT'] = 'seniorfemale'

In [ ]:
# numerik olmayan ve yalnızca iki eşsiz sınıfı olan column isimlerini getiriyoruz.
binary_cols = [col for col in df.columns if df[col].dtype not in [int, float]
               and df[col].nunique() == 2]

### Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
labelencoder = LabelEncoder()
for i in binary_cols:
    df[i] = labelencoder.fit_transform(df[i])

In [ ]:
def rare_analyser(dataframe, target, cat_cols):
    for col in cat_cols:
        print(col, ":", len(dataframe[col].value_counts()))
        print(pd.DataFrame({"COUNT": dataframe[col].value_counts(),
                            "RATIO": dataframe[col].value_counts() / len(dataframe),
                            "TARGET_MEAN": dataframe.groupby(col)[target].mean()}), end="\n\n\n")


def rare_encoder(dataframe, rare_perc):
    temp_df = dataframe.copy()

    rare_columns = [col for col in temp_df.columns if temp_df[col].dtypes == 'O'
                    and (temp_df[col].value_counts() / len(temp_df) < rare_perc).any(axis=None)]

    for var in rare_columns:
        tmp = temp_df[var].value_counts() / len(temp_df)
        rare_labels = tmp[tmp < rare_perc].index
        temp_df[var] = np.where(temp_df[var].isin(rare_labels), 'Rare', temp_df[var])

    return temp_df

In [ ]:
rare_analyser(df, "SURVIVED", cat_cols)
df = rare_encoder(df, 0.01)

In [ ]:
# eşsiz sınıf sayısı 10 ile 2 arasında olan column isimlerini getiriyoruz.
ohe_cols = [col for col in df.columns if 10 >= df[col].nunique() > 2]

In [ ]:
#One hot encoding
df=pd.get_dummies(df, columns=ohe_cols, drop_first=True)

In [ ]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)

In [ ]:
useless_cols = [col for col in df.columns if
                df[col].nunique() == 2 and (df[col].value_counts() / len(df) < 0.01).any(axis=None)]

In [ ]:
df.drop(useless_cols, axis=1, inplace=True)

### Standartlaştırma

In [ ]:
#Modele katmadan önce standartlaştırma işlemini uyguluyoruz.
scaler = StandardScaler()

In [ ]:
df[num_cols] = scaler.fit_transform(df[num_cols])

### Model Kurma

In [ ]:
#Bu bir sınıflandırma problemi olduğundan doğrusal sınıflandırma modellerinden biri olan logistic regressionu
# kullanıyoruz.
from sklearn.linear_model import LogisticRegression

In [ ]:
y = df["SURVIVED"]
X = df.drop(["PASSENGERID", "SURVIVED"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=46)

In [ ]:
log_model = LogisticRegression().fit(X_train, y_train)

### Kurulan Modelin Başarısını Ölçme

In [ ]:
#Eğittiğimiz veri setinin modelle tahmin üretmesini sağlıyoruz.
y_pred = log_model.predict(X_train)
accuracy_score(y_train, y_pred)

In [ ]:
#Tahminlerin 1-0 olma olasılıklarını getiriyor
y_prob = log_model.predict_proba(X_test)[:, 1]

# Test setinde modelimizi kontrol etme zamanı,tahmin üretiyoruz.
y_pred = log_model.predict(X_test)

In [ ]:
# ACCURACY,
#Doğru tahminlerin tüm tahminlere oranını verir.
accuracy_score(y_test, y_pred)

In [ ]:
# PRECISION
#Pozitif olarak tahmin edilenlerin doğru tahmin edilme oranlarını verir.
precision_score(y_test, y_pred)

In [ ]:
# RECALL
#Gerçekte pozitif olan durumların doğru tahmin edilme oranını verir.
recall_score(y_test, y_pred)

In [ ]:
f1_score(y_test, y_pred)

In [ ]:
# AUC
# roc eğrisinin altında kalan alanı verir.
roc_auc_score(y_test, y_prob)

In [ ]:
#Veee sonuç,sınıflandırma raporumuza eriştik.
print(classification_report(y_test, y_pred))

Sınıflandırma problemlerinde f1-score,auc score,recall ve precision başarı değerlendirme ölçütlerine bakıyoruz.1'e ne kadar yakınlarsa o kadar mutlu oluyoruz. :)
Görüşmek üzereeeee :)